https://katex.org/docs/supported.html

# OLP: Online Linear Programming



### Pyomo_0

First version of OLP using Pyomo.  
This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)


$$
\begin{array}{llllll}
\text{min} & \displaystyle\sum_{j=1}^{n=5} \left( e_{j}  -  x_{j} \right) & \\\\
\text{s.t.} & x_{j} \geq 0  &  \forall j = 1, \ldots, n\\
& x_{j} \leq ex_{j} & \forall j = 1, \ldots, n \\
& x_{j} \leq ev_{j} & \forall j = 1, \ldots, n\\
& x_{j} \leq se_{j} & \forall j = 1, \ldots, n\\ 
\end{array} 
$$



$$
\begin{array}{ll}
\text{j =  time slot}
\end{array} 
$$



#### Excel-model: results

<img src="./images/pyomo_o.png" width="1000">

<br>





<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0">
        <strong>Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!! 
    <BR>
    >  Check at the end the <strong>objective outcome</strong> and the <strong>Power to be supplied</strong>  
    <BR>
    <BR>
</div>

### Pyomo-model 0: Code & Results

In [23]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

from pyomo.environ import *

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.J = Set()  # set of time periods for a certain horizon (h=5)

# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

model.ex = Param(model.J)  # enexis max Power output for each time period (constant)
model.ev = Param(model.J)  # EV max Power input for each time period
model.se = Param(model.J)  # EVSE max Power output for each time period

# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------
model.x = Var(model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return sum(model.ex[j] - model.x[j] for j in model.J)


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev power input <= enexsis max power output?
def demand_constraint_rule(model, j):
    return model.x[j] <= model.ex[j]


# is ev power input <= ev max power input?
def ev_mpi_constraint_rule(model, j):
    return model.x[j] <= model.ev[j]


# is ev power input <= evse max power output?
def se_mpo_constraint_rule(model, j):
    return model.x[j] <= model.se[j]


# the next lines create one constraint for each member of the set model.J
model.EX_MPO_Constraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPI_Constraint = Constraint(model.J, rule=ev_mpi_constraint_rule)
model.SE_MPO_Constraint = Constraint(model.J, rule=se_mpo_constraint_rule)

# --------------------------------------------------------------------------
# Data
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/index.html
# --------------------------------------------------------------------------

n = 5

data = {
    None: {
        "J": {None: list(range(1, n + 1))},
        "ev": {
            (1): 7.36,
            (2): 7.36,
            (3): 5.00,
            (4): 3.68,
            (5): 0.0,
        },  # EV   max power input  per time period
        "se": {
            (1): 5.00,
            (2): 7.36,
            (3): 7.36,
            (4): 7.36,
            (5): 7.36,
        },  # EVSE max power output per time period
        "ex": {1: 14.72, 2: 14.72, 3: 14.72, 4: 14.72, 5: 14.72},
    }
}

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance(data)

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

tgc.pprint()
print("\n")
print(f"Power to be supplied: {[tgc.x[x].value for x in range(1, 6)]}")
print("\nObjective Outcome = ", value(tgc.OBJ))

1 Set Declarations
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

3 Param Declarations
    ev : Size=5, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :  7.36
          2 :  7.36
          3 :   5.0
          4 :  3.68
          5 :   0.0
    ex : Size=5, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 : 14.72
          2 : 14.72
          3 : 14.72
          4 : 14.72
          5 : 14.72
    se : Size=5, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   5.0
          2 :  7.36
          3 :  7.36
          4 :  7.36
          5 :  7.36

1 Var Declarations
    x : Size=5, Index=J
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   5.0 :  None : False : False : NonNegativeReals
          2 :     0 :  7.36 :  None : False : False : NonNegativeReals
      

### Pyomo_1

Extending the Pyomo_0 model above with additional constraints.   
This example relates to the ``pyomo_1`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)


$$
\begin{array}{lllll}
\text{min} & \displaystyle\sum_{j=1}^{n=5} \left( e_{j}  -  \displaystyle\sum_{i=1}^{m=3} x_{ij} \right) & \\\\
\text{s.t.} & \sum_{i=1}^{m=3} x_{ij} \leq e_{j} & \forall i = 1, \ldots, m \\\\
& x_{ij} \geq 0 & \forall j = 1, \ldots, n 
\end{array} 
$$



$$
\begin{array}{ll}
\text{i =  EVSE number}  \\
\text{j =  time slot}
\end{array} 
$$

### Pyomo-model 1: Code & Results

In [ ]:
# Tetris Game Charger

# from __future__ import division
from pyomo.environ import *

m = 3
n = 5

model = AbstractModel()

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)

v = {}
v[1, 1] = 5
v[2, 1] = 5
v[3, 1] = 5

# enexis max Power output for each time period (constant)
model.ex = Param(model.J, default=14.72)
# EV max Power input for each time period
model.ev = Param(model.I, model.J, default=0)
# EVSE max Power output for each time period
model.se = Param(model.I, model.J, initialize=v, default=7.36)

# the next line declares a variable indexed by the set J
model.x = Var(model.I, model.J, domain=NonNegativeReals)


def obj_expression(model):
    # return the expression for the objective
    return sum(model.ex[j] - sum(model.x[i, j] for i in model.I) for j in model.J)


model.OBJ = Objective(rule=obj_expression)


def demand_constraint_rule(model, j):
    # return the expression for the constraint for j
    return sum(model.x[i, j] for i in model.I) <= model.ex[j]


def ev_mpi_constraint_rule(model, i, j):
    # return the expression for the constraint for j
    return model.x[i, j] <= model.ev[i, j]


def se_mpo_constraint_rule(model, i, j):
    # return the expression for the constraint for j
    return model.x[i, j] <= model.se[i, j]


# the next line creates one constraint for each member of the set model.J
model.PwrDmdConstraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPIConstraint = Constraint(model.I, model.J, rule=ev_mpi_constraint_rule)
model.SE_MPOConstraint = Constraint(model.I, model.J, rule=se_mpo_constraint_rule)

# --------------------------------------------------------------------------
# data section

data = {
    None: {
        # "I": {None: list(range(1, m + 1))},
        # "J": {None: list(range(1, n + 1))},
        "ev": {
            (1, 1): 7.36,
            (1, 2): 7.36,
            (1, 3): 5.00,
            (1, 4): 3.68,
            (1, 5): 0.0,
            (2, 1): 7.36,
            (2, 2): 7.36,
            (2, 3): 5.00,
            (2, 4): 3.68,
            (2, 5): 0.0,
            (3, 1): 7.36,
            (3, 2): 7.36,
            (3, 3): 5.00,
            (3, 4): 3.68,
            (3, 5): 0.0,
        },  # EV   max power input  per time period
        # "se": {
        #     (1, 1): 7.36,
        #     (1, 2): 7.36,
        #     (1, 3): 7.36,
        #     (1, 4): 7.36,
        #     (1, 5): 7.36,
        #     (2, 1): 7.36,
        #     (2, 2): 7.36,
        #     (2, 3): 7.36,
        #     (2, 4): 7.36,
        #     (2, 5): 7.36,
        #     (3, 1): 7.36,
        #     (3, 2): 7.36,
        #     (3, 3): 7.36,
        #     (3, 4): 7.36,
        #     (3, 5): 7.36,
        # },  # EVSE max power output per time period
        # "ex": {1: 14.72, 2: 14.72, 3: 14.72, 4: 14.72, 5: 14.72},
    }
}


# --------------------------------------------------------------------------
# create a model instance and optimize

tgc = model.create_instance(data)

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
tgc.pprint()
print("\n")
print(f"Power to be supplied EVSE 1: {[tgc.x[1, x].value for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 2: {[tgc.x[2, x].value for x in range(1, 6)]}")
print(f"Power to be supplied EVSE 3: {[tgc.x[3, x].value for x in range(1, 6)]}")
print("\nObjective Outcome = ", value(tgc.OBJ))

### Pyomo-model 2: Code & Results